## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [19]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,63.00,98,100,8.01,overcast clouds
1,1,Altay,CN,47.8667,88.1167,41.79,28,3,4.45,clear sky
2,2,Tabuk,PH,17.4189,121.4443,84.04,52,86,3.15,overcast clouds
3,3,Lena,ES,43.1000,-5.7333,42.80,75,0,4.61,clear sky
4,4,Guerrero Negro,MX,27.9769,-114.0611,59.77,79,43,16.02,scattered clouds


In [20]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Tabuk,PH,17.4189,121.4443,84.04,52,86,3.15,overcast clouds
5,5,Rikitea,PF,-23.1203,-134.9692,78.37,72,8,6.73,light rain
6,6,Puerto Ayora,EC,-0.7393,-90.3518,76.15,88,51,3.04,broken clouds
13,13,Butaritari,KI,3.0707,172.7902,80.92,78,100,8.23,light rain
22,22,Georgetown,MY,5.4112,100.3354,84.20,83,20,4.61,few clouds
31,31,Vaini,TO,-21.2000,-175.2000,78.80,78,75,8.05,broken clouds
34,34,Weligama,LK,5.9667,80.4167,78.26,82,26,3.00,scattered clouds
36,36,Kavaratti,IN,10.5669,72.6420,83.17,69,11,3.56,few clouds
37,37,Bambous Virieux,MU,-20.3428,57.7575,75.99,94,40,3.44,scattered clouds
44,44,Samarai,PG,-10.6167,150.6667,82.04,60,17,10.02,few clouds


In [22]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Tabuk,PH,17.4189,121.4443,84.04,52,86,3.15,overcast clouds
5,5,Rikitea,PF,-23.1203,-134.9692,78.37,72,8,6.73,light rain
6,6,Puerto Ayora,EC,-0.7393,-90.3518,76.15,88,51,3.04,broken clouds
13,13,Butaritari,KI,3.0707,172.7902,80.92,78,100,8.23,light rain
22,22,Georgetown,MY,5.4112,100.3354,84.20,83,20,4.61,few clouds
31,31,Vaini,TO,-21.2000,-175.2000,78.80,78,75,8.05,broken clouds
34,34,Weligama,LK,5.9667,80.4167,78.26,82,26,3.00,scattered clouds
36,36,Kavaratti,IN,10.5669,72.6420,83.17,69,11,3.56,few clouds
37,37,Bambous Virieux,MU,-20.3428,57.7575,75.99,94,40,3.44,scattered clouds
44,44,Samarai,PG,-10.6167,150.6667,82.04,60,17,10.02,few clouds


In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Tabuk,PH,84.04,overcast clouds,17.4189,121.4443,
5,Rikitea,PF,78.37,light rain,-23.1203,-134.9692,
6,Puerto Ayora,EC,76.15,broken clouds,-0.7393,-90.3518,
13,Butaritari,KI,80.92,light rain,3.0707,172.7902,
22,Georgetown,MY,84.20,few clouds,5.4112,100.3354,
31,Vaini,TO,78.80,broken clouds,-21.2000,-175.2000,
34,Weligama,LK,78.26,scattered clouds,5.9667,80.4167,
36,Kavaratti,IN,83.17,few clouds,10.5669,72.6420,
37,Bambous Virieux,MU,75.99,scattered clouds,-20.3428,57.7575,
44,Samarai,PG,82.04,few clouds,-10.6167,150.6667,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Tabuk,PH,84.04,overcast clouds,17.4189,121.4443,Golden Berries Hotel and Convention
5,Rikitea,PF,78.37,light rain,-23.1203,-134.9692,Pension Maro'i
6,Puerto Ayora,EC,76.15,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
13,Butaritari,KI,80.92,light rain,3.0707,172.7902,Isles Sunset Lodge
22,Georgetown,MY,84.20,few clouds,5.4112,100.3354,Cititel Penang
31,Vaini,TO,78.80,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
34,Weligama,LK,78.26,scattered clouds,5.9667,80.4167,Weligama Bay Resort
36,Kavaratti,IN,83.17,few clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
37,Bambous Virieux,MU,75.99,scattered clouds,-20.3428,57.7575,Casa Tia Villa
44,Samarai,PG,82.04,few clouds,-10.6167,150.6667,Nuli Sapi


In [26]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Tabuk,PH,84.04,overcast clouds,17.4189,121.4443,Golden Berries Hotel and Convention
5,Rikitea,PF,78.37,light rain,-23.1203,-134.9692,Pension Maro'i
6,Puerto Ayora,EC,76.15,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
13,Butaritari,KI,80.92,light rain,3.0707,172.7902,Isles Sunset Lodge
22,Georgetown,MY,84.20,few clouds,5.4112,100.3354,Cititel Penang
...,...,...,...,...,...,...,...
611,Buala,SB,85.05,overcast clouds,-8.1450,159.5921,Maringe Lagoon Lodge
614,Noumea,NC,78.80,overcast clouds,-22.2763,166.4572,Hôtel Le Lagon
622,Las Cruces,US,77.00,overcast clouds,32.3123,-106.7783,Days End Lodge
623,Isangel,VU,86.00,broken clouds,-19.5500,169.2667,Volcano Sea View Tree House


In [28]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
locations.head(10)

,Lat,Lng
2,17.4189,121.4443
5,-23.1203,-134.9692
6,-0.7393,-90.3518
13,3.0707,172.7902
22,5.4112,100.3354
31,-21.2000,-175.2000
34,5.9667,80.4167
36,10.5669,72.6420
37,-20.3428,57.7575
44,-10.6167,150.6667


In [30]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))